<a href="https://colab.research.google.com/github/smiliyo55/psoCNN_Brain_Tumor/blob/main/particle1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install import_ipynb
import import_ipynb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
from copy import deepcopy

import utils1

import os
from torchsummary import summary
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader

importing Jupyter notebook from utils1.ipynb


In [ ]:
##### Model ####
class p_model(nn.Sequential):
  def __init__(self, list_layers, input_width, input_height, input_channels, output_dim, dropout_rate) :
    super(p_model, self).__init__()

    in_w = input_width
    in_h = input_height

    self.list_layers = list_layers

    self.model = nn.Sequential()

    for i in range(len(list_layers)):
        if list_layers[i]["type"] == "conv":
            n_out_filters = list_layers[i]["ou_c"]
            kernel_size = list_layers[i]["kernel"]

            if i == 0:
                in_c = input_channels

                p = int(( kernel_size - 1 ) / 2)
                if kernel_size % 2 == 0:
                  self.model.add_module(f'padding{i+1}', nn.ZeroPad2d((p, p+1, p, p+1)))
                  self.model.add_module(f'conv{i+1}', nn.Conv2d(
                                    in_channels=in_c,
                                    out_channels=n_out_filters,
                                    kernel_size=kernel_size,
                                    stride=1,
                                    bias=True
                                    ))
                  self.model.add_module(f'relu{i+1}', nn.ReLU(inplace=True))
                  self.model.add_module(f'batchnorm{i+1}', nn.BatchNorm2d(num_features=n_out_filters))
                else:
                  self.model.add_module(f'padding{i+1}', nn.ZeroPad2d((p, p, p, p)))
                  self.model.add_module(f'conv{i+1}', nn.Conv2d(
                                    in_channels=in_c,
                                    out_channels=n_out_filters,
                                    kernel_size=kernel_size,
                                    stride=1,
                                    bias=True
                                    ))
                  self.model.add_module(f'relu{i+1}', nn.ReLU(inplace=True))
                  self.model.add_module(f'batchnorm{i+1}', nn.BatchNorm2d(num_features=n_out_filters))
            else:
                in_c = list_layers[i-1]["ou_c"]

                #self.model.add_module(f'batchnorm{i+1}', nn.Dropout(dropout_rate))
                p = int(( kernel_size - 1 ) / 2)
                if kernel_size % 2 == 0:
                  self.model.add_module(f'padding{i+1}', nn.ZeroPad2d((p, p+1, p, p+1)))
                  self.model.add_module(f'conv{i+1}', nn.Conv2d(
                                    in_channels=in_c,
                                    out_channels=n_out_filters,
                                    kernel_size=kernel_size,
                                    stride=1,
                                    bias=True
                                    ))
                  self.model.add_module(f'relu{i+1}', nn.ReLU(inplace=True))
                  self.model.add_module(f'batchnorm{i+1}', nn.BatchNorm2d(num_features=n_out_filters))
                else:
                  self.model.add_module(f'padding{i+1}', nn.ZeroPad2d((p, p, p, p)))
                  self.model.add_module(f'conv{i+1}', nn.Conv2d(
                                    in_channels=in_c,
                                    out_channels=n_out_filters,
                                    kernel_size=kernel_size,
                                    stride=1,
                                    bias=True
                                    ))
                  self.model.add_module(f'relu{i+1}', nn.ReLU(inplace=True))
                  self.model.add_module(f'batchnorm{i+1}', nn.BatchNorm2d(num_features=n_out_filters))

        if list_layers[i]["type"] == "max_pool":
            kernel_size = list_layers[i]["kernel"]
            self.list_layers[i]["ou_c"] = list_layers[i-1]["ou_c"]

            self.model.add_module(f'maxpool{i+1}', nn.MaxPool2d(kernel_size=kernel_size, stride=2))

            in_w = int(in_w/2)
            in_h = int(in_h/2)
            #print(in_w, ", ", in_h)

        if list_layers[i]["type"] == "avg_pool":
            kernel_size = list_layers[i]["kernel"]
            self.list_layers[i]["ou_c"] = list_layers[i-1]["ou_c"]

            self.model.add_module(f'avgpool{i+1}', nn.AvgPool2d(kernel_size=kernel_size, stride=2))

            in_w = int(in_w/2)
            in_h = int(in_h/2)
            #print(in_w, ", ", in_h)

        if list_layers[i]["type"] == "fc":
          if list_layers[i-1]["type"] != "fc":
            self.model.add_module('flatten', nn.Flatten())

            if i == len(list_layers) - 1:
              in_c = list_layers[i-1]["ou_c"]
              in_f = in_c * in_w * in_h
              #print(in_f)
              self.model.add_module(f'linear{i+1}', nn.Linear(in_features=in_f, out_features=output_dim))
              #self.model.add_module(f'batchnorm{i+1}', nn.BatchNorm1d(num_features=output_dim))
              #self.model.add_module('softmax', nn.Softmax(dim=1))
            else:
              in_c = list_layers[i-1]["ou_c"]
              in_f = in_c * in_w * in_h
              #print(in_f)
              self.model.add_module(f'linear{i+1}', nn.Linear(in_features=in_f, out_features=list_layers[i]["ou_c"]))
              self.model.add_module(f'relu{i+1}', nn.ReLU(inplace=True))
              self.model.add_module(f'batchnorm{i+1}', nn.BatchNorm1d(num_features=list_layers[i]["ou_c"]))

          else :
            #self.model.add_module(f'dropout{i+1}', nn.Dropout(dropout_rate))

            if i == len(list_layers) - 1:
                self.model.add_module(f'linear{i+1}', nn.Linear(list_layers[i-1]["ou_c"], output_dim))
                #self.model.add_module(f'batchnorm{i+1}', nn.BatchNorm1d(num_features=output_dim))
                #self.model.add_module('softmax', nn.Softmax(dim=1))
            else:
                self.model.add_module(f'linear{i+1}', nn.Linear(list_layers[i-1]["ou_c"], list_layers[i]["ou_c"]))
                self.model.add_module(f'relu{i+1}', nn.ReLU(inplace=True))
                self.model.add_module(f'batchnorm{i+1}', nn.BatchNorm1d(num_features=list_layers[i]["ou_c"]))

In [ ]:
class Particle():
    def __init__(self, min_layer, max_layer, max_pool_layers, input_width, input_height, input_channels, \
        conv_prob, pool_prob, fc_prob, max_conv_kernel, max_out_ch, max_fc_neurons, output_dim):
        self.input_width = input_width
        self.input_height = input_height
        self.input_channels = input_channels

        self.num_pool_layers = 0
        self.max_pool_layers = max_pool_layers

        self.feature_width = input_width
        self.feature_height = input_height

        self.depth = np.random.randint(min_layer, max_layer)
        self.conv_prob = conv_prob
        self.pool_prob = pool_prob
        self.fc_prob = fc_prob
        self.max_conv_kernel = max_conv_kernel
        self.max_out_ch = max_out_ch

        self.max_fc_neurons = max_fc_neurons
        self.output_dim = output_dim

        self.layers = []
        self.acc = None
        self.vel = [] # Initial velocity
        self.pBest = []

        # Build particle architecture
        self.initialization()

        # Update initial velocity ??
        for i in range(len(self.layers)):
            if self.layers[i]["type"] != "fc":
                self.vel.append({"type": "keep"})
            else:
                self.vel.append({"type": "keep_fc"})

        self.model = None

        self.pBest = deepcopy(self)

        self.criterion = None

        self.optimizer = None

        if torch.cuda.is_available():
          self.device_name = torch.device("cuda")
        else:
          self.device_name = torch.device("cpu")


    def __str__(self):
        string = ""
        for z in range(len(self.layers)):
            string = string + self.layers[z]["type"] + " | "

        return string

    def initialization(self):
        out_channel = np.random.randint(3, self.max_out_ch)
        conv_kernel = np.random.randint(3, self.max_conv_kernel)

        # First layer is always a convolution layer
        self.layers.append({"type": "conv", "ou_c": out_channel, "kernel": conv_kernel})

        conv_prob = self.conv_prob
        pool_prob = self.pool_prob
        fc_prob = self.fc_prob

        for i in range(1, self.depth):
            if self.layers[-1]["type"] == "fc":
                layer_type = 1.1
            else:
                layer_type = np.random.rand()

            if layer_type < conv_prob:
                self.layers = utils1.add_conv(self.layers, self.max_out_ch, self.max_conv_kernel)

            elif layer_type >= conv_prob and layer_type <= pool_prob:
                self.layers, self.num_pool_layers = utils1.add_pool(self.layers, self.fc_prob, self.num_pool_layers,
                                                                   self.max_pool_layers, self.max_out_ch,
                                                                   self.max_conv_kernel, self.max_fc_neurons,
                                                                   self.output_dim)

            elif layer_type >= fc_prob:
                self.layers = utils1.add_fc(self.layers, self.max_fc_neurons)

        self.layers[-1] = {"type": "fc", "ou_c": self.output_dim, "kernel": -1}


    def velocity(self, gBest, Cg):
        self.vel = utils1.computeVelocity(gBest, self.pBest.layers, self.layers, Cg)

    def update(self):
        new_p = utils1.updateParticle(self.layers, self.vel)
        new_p = self.validate(new_p)

        self.layers = new_p
        self.model = None

    def validate(self, list_layers):
        # Last layer should always be a fc with number of neurons equal to the number of outputs
        list_layers[-1] = {"type": "fc", "ou_c": self.output_dim, "kernel": -1}

        # Remove excess of Pooling layers
        self.num_pool_layers = 0
        for i in range(len(list_layers)):
            if list_layers[i]["type"] == "max_pool" or list_layers[i]["type"] == "avg_pool":
                self.num_pool_layers += 1

                if self.num_pool_layers >= self.max_pool_layers:
                    list_layers[i]["type"] = "remove"


        # Now, fix the inputs of each conv and pool layers
        updated_list_layers = []

        for i in range(0, len(list_layers)):
            if list_layers[i]["type"] != "remove":
                if list_layers[i]["type"] == "conv":
                    updated_list_layers.append({"type": "conv", "ou_c": list_layers[i]["ou_c"],
                                                "kernel": list_layers[i]["kernel"]})

                if list_layers[i]["type"] == "fc":
                    updated_list_layers.append(list_layers[i])

                if list_layers[i]["type"] == "max_pool":
                    updated_list_layers.append({"type": "max_pool", "ou_c": list_layers[i-1]["ou_c"], "kernel": 2})

                if list_layers[i]["type"] == "avg_pool":
                    updated_list_layers.append({"type": "avg_pool", "ou_c": list_layers[i-1]["ou_c"], "kernel": 2})

        return updated_list_layers

    ##### Model methods ####
    def model_compile(self, dropout_rate):
      self.model = p_model( list_layers=self.layers, input_width=self.input_width, input_height=self.input_height,
                           input_channels=self.input_channels, output_dim=self.output_dim, dropout_rate=dropout_rate)

      self.layers = self.model.list_layers

      learning_rate = 0.001
      self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)

      self.criterion = nn.CrossEntropyLoss()
      self.model.to(self.device_name)


    def model_summary(self):
      print(self.model)
      summary(self.model, (self.input_channels, self.input_height, self.input_width), 16, self.device_name.type)


    def model_fit(self, dataloader, batch_size, epochs):
      train_loss_list = []
      train_acc_list = []

      for epoch in range(epochs):
          self.model.train()
          running_loss = 0.0
          correct = 0
          total = 0

          for i, data in enumerate(dataloader, 0):
              inputs, labels = data
              inputs, labels = inputs.to(self.device_name), labels.to(self.device_name)
              self.optimizer.zero_grad()

              outputs = self.model(inputs)
              loss = self.criterion(outputs, labels)
              loss.backward()
              self.optimizer.step()

              running_loss += loss.item()

              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

          epoch_loss = running_loss / len(dataloader)
          epoch_acc = correct / total

          train_loss_list.append(epoch_loss)
          train_acc_list.append(epoch_acc)

      return {'loss': train_loss_list, 'accuracy': train_acc_list}

    def evaluate(self, dataloader, batch_size):
      self.model.eval()
      test_loss = 0.0
      correct = 0
      total = 0

      with torch.no_grad():
          for inputs, labels in dataloader:
              inputs, labels = inputs.to(self.device_name), labels.to(self.device_name)
              outputs = self.model(inputs)
              loss = self.criterion(outputs, labels)
              test_loss += loss.item()

              _, predicted = torch.max(outputs, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      accuracy = correct / total
      average_loss = test_loss / len(dataloader)

      return {'loss': average_loss, 'accuracy': accuracy}

    def model_fit_complete(self, dataloader1, dataloader2, batch_size, epochs):
        train_losses = []
        val_losses = []
        train_accuracies = []
        val_accuracies = []

        for epoch in range(epochs):
          #training
          self.model.train()
          running_loss = 0.0
          correct = 0
          total = 0

          for i, data in enumerate(dataloader1, 0):
              inputs, labels = data
              inputs, labels = inputs.to(self.device_name), labels.to(self.device_name)
              self.optimizer.zero_grad()

              outputs = self.model(inputs)
              loss = self.criterion(outputs, labels)
              loss.backward()
              self.optimizer.step()

              running_loss += loss.item()

              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

          epoch_loss = running_loss / len(dataloader1)
          epoch_acc = correct / total

          train_losses.append(epoch_loss)
          train_accuracies.append(epoch_acc)

          # Validation
          self.model.eval()
          test_loss = 0.0
          correct = 0
          total = 0

          with torch.no_grad():
              for inputs, labels in dataloader2:
                  inputs, labels = inputs.to(self.device_name), labels.to(self.device_name)
                  outputs = self.model(inputs)
                  loss = self.criterion(outputs, labels)
                  test_loss += loss.item()

                  _, predicted = torch.max(outputs, 1)
                  total += labels.size(0)
                  correct += (predicted == labels).sum().item()

          accuracy = correct / total
          average_loss = test_loss / len(dataloader2)

          val_losses.append(average_loss)
          val_accuracies.append(accuracy)

          print(f'Epoch [{epoch+1}/{epochs}], '
          f'Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.2%}, '
          f'Validation Loss: {average_loss:.4f}, Validation Accuracy: {accuracy:.2%}')

        return {'train loss': train_losses, 'train accuracy': train_accuracies, 'val loss': val_losses, 'val accuracy': val_accuracies}


    def model_delete(self):
      del self.model
      torch.cuda.empty_cache()
      self.model = None

In [ ]:
#test = Particle(min_layer=3, max_layer=10, max_pool_layers=4, input_width=28, input_height=28, input_channels=1, \
#                conv_prob=0.2, pool_prob=0.3, fc_prob=0.4, max_conv_kernel=4, max_out_ch=16, max_fc_neurons=3*28*28, output_dim=4)
#test.layers

In [ ]:
#test.model_compile(dropout_rate=0.5)

In [ ]:
#test.model

In [ ]:
def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    return sum(params)

In [ ]:
#count_parameters(test.model)